In [10]:
percent_of_test_data = 30

In [11]:
import sys
sys.path.append('/Users/tomek/OneDrive/Projects/Machine Learning/passat-b6-tdi')

import urllib.request
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from bs4 import BeautifulSoup
import predict


In [12]:
main_url = 'https://www.otomoto.pl/osobowe/volkswagen/passat/b6-2005-2010/?search%5Bfilter_enum_fuel_type%5D%5B0%5D=diesel&search%5Bfilter_enum_damaged%5D=0&search%5Bcountry%5D='
html = urllib.request.urlopen(main_url).read()
soup = BeautifulSoup(html,"lxml")

pages = int(soup.find_all('span', attrs={'class': 'page'})[-2].text)
print('Found pages: ',pages)

Found pages:  51


In [13]:
Xy = predict.collect_data(main_url)

Page: 1 | Offers: 32


Page: 2 | Offers: 32


Page: 3 | Offers: 32


Page: 4 | Offers: 32


Page: 5 | Offers: 32


Page: 6 | Offers: 32


Page: 7 | Offers: 32


Page: 8 | Offers: 32


Page: 9 | Offers: 32


In [14]:
print(Xy.shape)
print(Xy[-10:])

X_train, y_train, X_test, y_test = predict.split_data(Xy, percent_of_test_data) 

#print(X.shape)
#print(y.shape)

(254, 4)
[[   2007.  268500.    1896.   22900.]
 [   2006.  275000.    1968.   13000.]
 [   2007.  260000.    1968.   21900.]
 [   2008.  191200.    1968.   21900.]
 [   2007.  136000.    1968.   18900.]
 [   2006.  390694.    1968.   19900.]
 [   2007.  287000.    1896.   19800.]
 [   2009.  170000.    1968.   32800.]
 [   2006.  311000.    1968.   19100.]
 [   2006.  235000.    1896.   20900.]]

Total samples:  254
Training samples: 178
Test samples: 76


In [15]:
regr = linear_model.LinearRegression(normalize=True)
regr.fit(X_train, y_train)

np.set_printoptions(formatter={'float_kind':'{:f}'.format})
print('Interceptor: ',regr.intercept_)
print('Coefficients: ', regr.coef_)

Interceptor:  [-5052798.184726]
Coefficients:  [[2529.514178 0.000122 -0.985463]]


In [16]:
y_pred = regr.predict(X_test)

#print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Variance score: 0.45


In [17]:
price_pred = regr.predict([[2010,180000, 1968]])
print('The best price for your Passat B6 is '+str(round(price_pred[0][0],2))+' PLN')

The best price for your Passat B6 is 29607.83 PLN
